<a href="https://colab.research.google.com/github/simonebugo/Big_Data/blob/main/MIL_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dowload the data from google drive

In [1]:
!pip install gdown
import gdown
FILE_ID = "12gAeqkWtmIM6HU7LSg3T_2t23vFx4sfm"
url = f'https://drive.google.com/uc?id={FILE_ID}'
output = 'archive.tar.gz'
gdown.download(url, output, quiet=False)

# Extract it
import tarfile, os
os.makedirs('data', exist_ok=True)
with tarfile.open('archive.tar.gz', 'r:gz') as tar:
    tar.extractall('data')

print("File downloaded and extracted!")

FileURLRetrievalError: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=12gAeqkWtmIM6HU7LSg3T_2t23vFx4sfm

but Gdown can't. Please check connections and permissions.

# Multiple Instance Learning (MIL) Lab

In this lab, you'll implement a **Multiple Instance Learning (MIL)** pipeline in PyTorch.  
You’ll work on:
- Creating a dataset that groups instances into *bags*.
- Implementing a **Attention** MIL model.

In [ ]:
#occorre installare la libreria di openslide nel mio ambiente di lavoro prima di farne l'import (aggiunta da me siccome notebook non andava)
!apt-get install openslide-tools
!pip install openslide-python

In [ ]:
import os
import glob
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import openslide
import h5py


## ⚙️ Configuration

This section defines global variables, file paths, and hyperparameters.  
You don’t need to modify this section — just review it to understand how the parameters control training.

In [ ]:
# ---------------------- Config / Hyperparameters ----------------------
DATA_DIR = "./pt_files"     # folder containing .pt files. directory in cui si trovano i file di dati di input, ovvero i file .pt che contengono gli embeddings pre-estratti.
EMBED_DIM = 1024            # dimension of UNI embeddings. è la dimensione delle feature (embedding) estratte da ciascuna patch dal modello UNI. in pratica UNI è il modello utilizzato per estrarre le feature da ciascuna patch di una WSI
PATCH_SIZE = 512           # size of each patch (in pixels). La dimensione (in pixel) della patch originale estratta dalla WSI (in questo caso 512 \times 512 pixel)
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(DEVICE)

NUM_EPOCHS = 2
BATCH_SIZE = 8 #Il numero di borse (bags) che vengono elaborate contemporaneamente in ogni passaggio di addestramento
HIDDEN_DIM = 256
LR = 1e-4 #è il learning rate
WEIGHT_DECAY = 1e-5 #termine di regolarizzazione applicato all'ottimizzazione
NUM_WORKERS = 4 #numero processi paralleli per cariare i dati da dsco al data loader


# Reproducibility
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

cuda


## Implement the MIL Dataset

In Multiple Instance Learning, data is organized into *bags*, each containing several *instances*.  
Each bag has a label (positive or negative), but the individual instances do not.

- Implement a custom PyTorch `Dataset` that loads instances and groups them into bags.

- Each `__getitem__` call should return:
  - `bag_tensor`: a tensor of shape `(num_instances, feature_dim)`
  - `label`: an integer (0 or 1)
  - `name`: the name of the sample


In [ ]:
class EmbeddingBagDataset(Dataset):
    def __init__(self, data_dir):
        """
        Args:
            data_dir (str): Directory containing features for each bag.
        """
        # TODO:
        # self.file_list = ...
        # Trova tutti i file .pt
        self.file_list = glob.glob(os.path.join(data_dir, '*.pt'))

        # self.items = list of (file_path, label) tuples
        # Estrae il nome base e l'etichetta per ogni file
        self.items = []
        for file_path in self.file_list:
            filename = os.path.basename(file_path) #Estrae solo il nome del file dal percorso completo
            label = infer_label_from_filename(filename) #Questa funzione legge la stringa filename e restituisce 0 se contiene 'normal' o 1 se contiene 'tumor'
            self.items.append((file_path, label)) #Aggiunge una tupla alla lista self.items. La tupla contiene il percorso completo del file (file_path) e la sua etichetta binaria (label)

        # self.labels = ...
        # Estrae le etichette per lo split (usato fuori dalla classe)
        self.labels = [label for (_path, label) in self.items]

    def __len__(self):
        # TODO: Return the total number of bags
        return len(self.items)

    def __getitem__(self, idx):
        # TODO:
        # Load the bag data from the corresponding .pt file
        file_path, label = self.items[idx]
        #
        # Returns:
        #   bag_tensor: torch.FloatTensor
        # Ogni file .pt contiene un tensore di forma (n_patches, 1024)]
        bag_tensor = torch.load(file_path, map_location=torch.device('cpu')).float()
        #   label: torch.LongTensor
        # Assicurati che l'etichetta sia un tensore Long
        label_tensor = torch.LongTensor([label])
        #   name: str (filename without extension)
        # Estrae il nome del file senza estensione (per la visualizzazione WSI)
        name = os.path.splitext(os.path.basename(file_path))[0]
        return bag_tensor, label_tensor, name



def infer_label_from_filename(fname):
    # filenames normal_XXXX.pt are negative (0), tumor_XXXX.pt are positive (1)
    if 'normal' in fname:
        return 0
    elif 'tumor' in fname:
        return 1


"""
spiegazione maschera: Quando il modello AB-MIL calcola l'attenzione, l'obiettivo è trovare le patch più importanti (quelle tumorali)].
Se calcoli l'attenzione su tutto il tensore paddato, il modello tratterà le righe di zeri del padding come se fossero patch di tessuto reale.
Questi zeri influenzerebbero (e distorcerebbero) in modo significativo il calcolo dell'attenzione e la media].
La maschera agisce come un "interruttore" per dire al modello: "Ignora questi elementi (gli zeri del padding)"].
La maschera è un tensore booleano bidimensionale di forma (batch_size, max_n) costruito come segue:
Valore True (1): Indica una posizione in cui è presente una vera patch di dati]. L'attenzione deve essere calcolata qui.
Valore False (0): Indica una posizione dove è stato aggiunto lo zero padding]. L'attenzione non deve essere calcolata qui.
"""

def collate_fn(batch):
    """
    Args:
        batch: list of tuples [(bag_tensor, label, name), ...]
    Returns:
        padded: torch.FloatTensor of shape (batch_size, max_n, emb_dim)
        mask: torch.BoolTensor of shape (batch_size, max_n)
        labels: torch.LongTensor of shape (batch_size,)
        names: list of str of length batch_size
    """
    # batch is a list of (bag_tensor, label, name) tuples
    # the number of patches in each bag may vary and pytorch will not like it
    # so we need to pad the bags to the same length
    bags, labels, names = zip(*batch)
    # TODO: figure out max number of patches per bag, and embedding dimension (should be EMBED_DIM, can be extracted from a bag)
    max_n = max(b.shape[0] for b in bags)
    emb_dim = bags[0].shape[1]  # Dovrebbe essere 1024, estraiamo dal primo elemento
    padded = torch.zeros(len(bags), max_n, emb_dim) # Inizializza i tensori di output con zeri
    mask = torch.zeros(len(bags), max_n, dtype=torch.bool) # Inizializza la maschera booleana a False (0) per il padding]
    # TODO: into padded copy each bag, into mask put 1s where there is data (it will be used to ignore the padded parts when computing attention)
    for i, b in enumerate(bags):
        n_patches = b.shape[0]
        # Copia il tensore originale nel tensore paddato
        padded[i, :n_patches, :] = b
        # Imposta la maschera a True (1) per tutti gli elementi che contengono dati reali
        mask[i, :n_patches] = True
    labels = torch.stack(labels).long()
    return padded, mask, labels, list(names)


In [ ]:
# Attention Based MIL --> https://arxiv.org/pdf/1802.04712
# find the formula, be careful on the activation functions used

"""
la formula per la Gated Attention è:
𝑎=softmax(𝐰^𝑇(tanh(𝐕𝐡^𝑇𝑘) ⊙ 𝜎(𝐔𝐡^𝑇𝑘)))
dove  𝐡𝑘  è l'embedding di una patch,
𝐕  e  𝐔  sono le trasformazioni lineari,
tanh  e  𝜎  (Sigmoid) sono le non-linearità,
⊙  è il prodotto elemento per elemento,
e  𝐰  proietta il risultato a 1 dimensione (il punteggio di attenzione grezzo).
"""

class GatedAttention(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super().__init__()
        self.V = nn.Linear(in_dim, hidden_dim)
        self.U = nn.Linear(in_dim, hidden_dim)
        self.w = nn.Linear(hidden_dim, 1)


    def forward(self, x, mask=None):
        Vx = torch.tanh(self.V(x))
        Ux = torch.sigmoid(self.U(x))
        H = Vx * Ux
        scores = self.w(H)


        if mask is not None:
            #se una maschera è fornita, imposta i punteggi degli elementi di padding a -inf
            # La maschera deve essere invertita (~) perché True indica i dati, False il padding.
            # Vogliamo che dove la maschera è False, il punteggio sia -inf. in modo che non contribusicano come valori tramite il softmax in quanto = 0
            scores = scores.masked_fill(~mask, float('-inf'))

        attn = torch.softmax(scores, dim=1)
        return attn


class ABMIL(nn.Module):
    def __init__(self, emb_dim, hidden_dim=256, n_classes=2, dropout=0.25):
        super().__init__()
        self.attention = GatedAttention(emb_dim, hidden_dim)
        # TODO: define a classifier (MLP) that takes as input the bag representation and outputs logits for n_classes
        self.classifier = nn.Sequential(
            nn.Linear(emb_dim, emb_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(emb_dim, n_classes) # Output i logits per la CrossEntropyLoss
        )


    def forward(self, x, mask=None):
        attn = self.attention(x, mask=mask)
        bag = torch.sum(attn.unsqueeze(-1) * x, dim=1)
        logits = self.classifier(bag)
        return logits, attn

### metrics, train and test epoch offered by our friend ChatGPT

In [ ]:
def compute_metrics(y_true, y_pred_probs, threshold=0.5):
    y_pred = (y_pred_probs >= threshold).astype(int)
    acc = accuracy_score(y_true, y_pred)
    try:
        roc = roc_auc_score(y_true, y_pred_probs)
    except Exception:
        roc = float('nan')
    prec, recall, _ = precision_recall_curve(y_true, y_pred_probs)
    aupr = auc(recall, prec)
    cm = confusion_matrix(y_true, y_pred)
    return {"accuracy": acc, "roc_auc": roc, "aupr": aupr, "confusion_matrix": cm}

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    losses = []
    all_probs = []
    all_labels = []
    for x, mask, y, _names in tqdm(loader):
        x, mask, y = x.to(device), mask.to(device), y.to(device)
        optimizer.zero_grad()
        logits, _attn = model(x, mask=mask)
        loss = criterion(logits, y)
        probs = F.softmax(logits, dim=1)[:, 1].detach().cpu().numpy()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        all_probs.extend(probs.tolist())
        all_labels.extend(y.detach().cpu().numpy().tolist())
    metrics = compute_metrics(np.array(all_labels), np.array(all_probs))
    return np.mean(losses), metrics

def eval_epoch(model, loader, criterion, device):
    model.eval()
    losses = []
    all_probs = []
    all_labels = []
    all_attn = []
    names_list = []
    with torch.no_grad():
        for x, mask, y, names in tqdm(loader):
            x, mask, y = x.to(device), mask.to(device), y.to(device)
            logits, attn = model(x, mask=mask)
            loss = criterion(logits, y)
            probs = F.softmax(logits, dim=1)[:, 1].cpu().numpy()
            losses.append(loss.item())
            all_probs.extend(probs.tolist())
            all_labels.extend(y.detach().cpu().numpy().tolist())
            all_attn.extend(attn.cpu().numpy().tolist())
            names_list.extend(names)
    metrics = compute_metrics(np.array(all_labels), np.array(all_probs))
    return np.mean(losses), metrics, all_attn, names_list



In [ ]:
dataset = EmbeddingBagDataset(DATA_DIR)
labels = np.array([lab for (_p, lab) in dataset.items])
indices = np.arange(len(dataset))


# split dataset into train, val, test
train_idx, temp_idx = train_test_split(indices, test_size=0.30, stratify=labels, random_state=SEED)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=labels[temp_idx], random_state=SEED)
train_dataset = torch.utils.data.Subset(dataset, train_idx)
val_dataset = torch.utils.data.Subset(dataset, val_idx)
test_dataset = torch.utils.data.Subset(dataset, test_idx)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"Dataset sizes: Train={len(train_dataset)}, Val={len(val_dataset)}, Test={len(test_dataset)}")

In [ ]:
# ---------------------- Build model ----------------------
model = ABMIL(emb_dim=EMBED_DIM, hidden_dim=HIDDEN_DIM, n_classes=2).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

In [ ]:
# ---------------------- Training loop ----------------------
best_val_auc = 0.0
for epoch in range(1, NUM_EPOCHS + 1):
    print(f"Epoch {epoch}/{NUM_EPOCHS}")
    train_loss, train_metrics = train_epoch(model, train_loader, criterion, optimizer, DEVICE)

    val_loss, val_metrics, val_attn, val_names = eval_epoch(model, val_loader, criterion, DEVICE)
    print(f"Epoch {epoch}/{NUM_EPOCHS} | Train loss: {train_loss:.4f} | Val loss: {val_loss:.4f} | Val AUROC: {val_metrics['roc_auc']:.4f}")
    if val_metrics["roc_auc"] > best_val_auc:
        best_val_auc = val_metrics["roc_auc"]
        torch.save({"model_state": model.state_dict(), "EMBED_DIM": EMBED_DIM}, "abmil_best.pth")
        print("Saved best model")

In [ ]:
ckpt = torch.load("abmil_best.pth", map_location=DEVICE)
model.load_state_dict(ckpt["model_state"])
test_loss, test_metrics, test_attn, test_names = eval_epoch(model, test_loader, criterion, DEVICE)
print("Test metrics:", test_metrics)

# Plot the WSIs and show high attention patches

In [ ]:
def extract_samples_from_datasets(WSIs, datasets):
    # figure out where the WSIs ended up (train/val/test)
    # TODO: return a list of (bag_tensor, label, name) for each WSI found in any of the datasets
    samples = []

    # Prepara un set di nomi WSI senza estensione (ad esempio, 'normal_002', 'tumor_002')
    wsi_names_to_find = {os.path.splitext(f)[0] for f in WSIs}

    # Itera su tutti i subset (train, val, test)
    for dataset_subset in datasets:
        # Quando si itera su un torch.utils.data.Subset, si ottengono solo gli indici.
        # Dobbiamo accedere all'oggetto dataset sottostante per recuperare i dati completi.

        # 1. Ottiene il dataset sottostante (EmbeddingBagDataset)
        # Se l'oggetto è un Subset, estrai l'attributo 'dataset'
        if isinstance(dataset_subset, torch.utils.data.Subset):
            base_dataset = dataset_subset.dataset
            indices = dataset_subset.indices
        else:
            base_dataset = dataset_subset
            indices = range(len(base_dataset))

        # 2. Itera sugli indici del subset e verifica se il nome corrisponde
        for idx in indices:
            # L'EmbeddingBagDataset (base_dataset) è la fonte di self.items
            file_path, label = base_dataset.items[idx]
            name = os.path.splitext(os.path.basename(file_path))[0]

            if name in wsi_names_to_find:
                # Carica la borsa completa (perché serve l'attention in seguito)
                bag_tensor, label_tensor, name = base_dataset[idx]

                # Rimuoviamo il nome per non cercarlo due volte (se necessario)
                # wsi_names_to_find.remove(name)

                samples.append((bag_tensor, label_tensor.item(), name)) # Convertiamo l'etichetta in int
    return samples

# Visualize attention for the WSIs
WSIs = os.listdir('WSI')

## NB, with random split the 2 patches for which we have WSI images may not be in test set, so we search in all subsets
# You could also split the dataset to make sure these 2 are in test set, which would be cleaner (extra TODO if you are bored)
samples = extract_samples_from_datasets(WSIs, [train_dataset, val_dataset, test_dataset])



## For each sample for which we have the WSI, extract the top patches with higher attention

In [ ]:
# ---------------------- Attention visualization ----------------------
def top_k_instances(attn_weights, top_k=10):
    # TODO: find out the indices of the top-k highest attention weights
    # 1. Trova gli indici ordinati dei pesi (dal più piccolo al più grande)
    sorted_indices = np.argsort(attn_weights)
    # 2. Inverti l'ordine per avere gli indici dal più grande al più piccolo
    top_indices_descending = sorted_indices[::-1]
    # 3. Seleziona i primi top_k indici
    top_k_instances = top_indices_descending[:top_k]
    return top_k_instances


model.eval()
k = 5
high_attention_patches = {}
with torch.no_grad():
    for sample_bag, sample_label, sample_name in samples:
        x = sample_bag.unsqueeze(0).to(DEVICE)
        mask = torch.ones(1, x.size(1), dtype=torch.bool).to(DEVICE)
        logits, attn = model(x, mask=mask)
        probs = F.softmax(logits, dim=1)[:, 1].cpu().numpy()
        attn = attn.squeeze(0).cpu().numpy()
        # get the top-k high attention patch indices
        top_k = top_k_instances(attn, top_k=k)
        high_attention_patches[sample_name] = top_k

## Figure out the coordinates of the high attention patches

In [ ]:
import os
import h5py
patch_dir = "./patches/"
patches = os.listdir(patch_dir)

for patch in patches:
    patch_path = os.path.join(patch_dir, patch)
    with h5py.File(patch_path, 'r') as f:
        coords = f['coords'][:]
    # coords is a numpy array of shape (num_patches, 2) with (x, y) coordinates of each patch
    patch_name = os.path.basename(patch).split('.')[0]
    top_k_coords = coords[high_attention_patches[patch_name]]
    WSI = f"./WSI/{patch_name}.tif"
    slide = openslide.OpenSlide(WSI)
    level_0_dim = slide.level_dimensions[0]
    print(f"Level 0 dimensions: {level_0_dim}")

    # Create an empty mask of the size of level 0
    mask = np.zeros((level_0_dim[1], level_0_dim[0]), dtype=np.uint8)
    for coord in top_k_coords:
        x_level0, y_level0 = coord
        # TODO: set mask to 1 in the region corresponding to the patch (x_level0, y_level0) is the top-left corner of the patch of size PATCH_SIZE
        # Imposta a 1 la regione corrispondente alla patch
        # Usiamo np.clip per evitare che le coordinate vadano fuori dai limiti della maschera
        x_end = np.clip(x_level0 + PATCH_SIZE, 0, level_0_dim[0])
        y_end = np.clip(y_level0 + PATCH_SIZE, 0, level_0_dim[1])
        mask[y_level0:y_end, x_level0:x_end] = 1

    # Read WSI at a lower level for visualization otherwise if you try to plot it the image will be too large
    x = 4
    level_x = slide.read_region((0, 0), x, slide.level_dimensions[x])
    # downsample the mask so that it matches level x (each level is downsampled by a factor of 2)
    downsample_factor = 2**x

    # Ridimensiona la maschera utilizzando solo la parte rilevante (i pixel a 1)
    from skimage.transform import downscale_local_mean
    # Esegue il downsampling della maschera
    mask_downsample = downscale_local_mean(mask, (downsample_factor, downsample_factor)) > 0
    mask_downsample = mask_downsample.astype(np.uint8)
    print(f"Mask downsampled shape: {mask_downsample.shape}, Level {x} dimensions: {slide.level_dimensions[x]}")
    # The image will still be too large and the patches will be small, only plot the part of the image where there are high attention patches
    # TODO: find bounding box of the high attention patches in level x coordinates
    # 1. Trova le coordinate minime e massime in Livello 0
    min_x_l0 = top_k_coords[:, 0].min()
    min_y_l0 = top_k_coords[:, 1].min()
    max_x_l0 = top_k_coords[:, 0].max() + PATCH_SIZE
    max_y_l0 = top_k_coords[:, 1].max() + PATCH_SIZE

    # 2. Calcola il fattore di downsampling
    downsample_factor = 2**x # lo usiamo qui (o slide.level_downsamples[x] se disponibile)

    # 3. Trasforma le coordinate in Livello X
    # Applichiamo un margine per vedere il contesto
    margin = 50

    xmin = np.clip((min_x_l0 // downsample_factor) - margin, 0, level_x.width)
    ymin = np.clip((min_y_l0 // downsample_factor) - margin, 0, level_x.height)
    xmax = np.clip((max_x_l0 // downsample_factor) + margin, 0, level_x.width)
    ymax = np.clip((max_y_l0 // downsample_factor) + margin, 0, level_x.height)

    # Assicurati che xmax > xmin e ymax > ymin per evitare errori di crop
    if xmax <= xmin: xmax = xmin + 1
    if ymax <= ymin: ymax = ymin + 1


    level_x = level_x.crop((xmin, ymin, xmax, ymax))
    mask_downsample = mask_downsample[ymin:ymax, xmin:xmax]
    # display mask as red overlay on level x, with some transparency
    plt.imshow(level_x)
    plt.imshow(mask_downsample, cmap='Reds', alpha=0.5)
    plt.axis('off')
    plt.show()
